# Scale calculations, descriptives and creation of a BIDS style participants.tsv file

Master Thesis: **Modulation of the Action Observation Network - Investigating Action, Actor and Observer Factors in an fMRI Study**

Author: Olaf Borghi olafborghi@gmail.com

### Setup

Install and import libraries

In [2]:
# import modules
import os
import numpy as np # math and arrays
import pandas as pd # data frames and data analysis
from scipy import stats # statistical tests
import researchpy as rp # more statistical tests
from matplotlib import pyplot as plt # plotting
from pathlib import Path # flexible path handling

Load data

Select a local file, in this case the xlsx file containing the data. I stored the datafile in the same folder as this script, making it easy to access across platforms.

In [3]:
# Fixed paths
data_dir = Path('/home/olafb99/mnt/p/userdata/olafb99/shared/03_AON_ACTION/02_scripts/')

In [4]:
data_path = data_dir / "data_evobrain.csv"
print(data_path)

/home/olafb99/mnt/p/userdata/olafb99/shared/03_AON_ACTION/02_scripts/data_evobrain.csv


In [5]:
# load the file
data_raw = pd.read_csv(data_path, sep = ";") 
data_raw.head()

,pn_code,age,gender,handedness,QCAE_1,QCAE_2,QCAE_3,QCAE_4,QCAE_5,QCAE_6,...,QCAE_31,dog_exp_1,dog_exp_2,dog_exp_3,dog_exp_3_opt1,dog_exp_4,dog_exp_4_opt1,dog_exp_5,dog_exp_5_opt1,dog_exp_5_opt2
0,EV01,19,1,1,3,3,2,1,2,2,...,2,4,3,2,NaN,2,NaN,2,NaN,NaN
1,EV02,21,2,1,4,3,1,2,2,2,...,3,3,2,2,NaN,2,NaN,2,NaN,NaN
2,EV03,26,2,1,4,2,1,1,1,1,...,1,4,3,2,NaN,2,NaN,2,NaN,NaN
3,EV04,20,1,1,3,3,2,2,2,3,...,2,5,4,1,20.0,1,1.0,1,3.0,Hundeschule
4,EV05,20,1,1,4,3,2,1,1,3,...,2,4,3,2,NaN,2,NaN,2,NaN,NaN


Recode items and variables

In [7]:
data_recoded = data_raw

# recode all QCAE items
print("first QCAE item: ", data_recoded.columns.get_loc("QCAE_1"))
print("last QCAE item: ", data_recoded.columns.get_loc("QCAE_31"))

qcae_cols = data_recoded.iloc[:,4:35]
for col in qcae_cols:
    data_recoded[col].replace({1:4, 2:3, 3:2, 4:1}, inplace=True) 

# a few items QCAE items have to be re-inverted 
# QCAE items 1, 2, 17 and 29 
data_recoded['QCAE_1'].replace({4:1, 3:2, 2:3, 1:4}, inplace=True) 
data_recoded['QCAE_2'].replace({4:1, 3:2, 2:3, 1:4}, inplace=True)
data_recoded['QCAE_17'].replace({4:1, 3:2, 2:3, 1:4}, inplace=True)
data_recoded['QCAE_29'].replace({4:1, 3:2, 2:3, 1:4}, inplace=True)

# recode factor levels
data_recoded['gender'].replace({1:0, 2:1, 3:2}, inplace=True) # recode to 0 = woman, 1 = man 2 = other
data_recoded['dog_exp_3'].replace({1:1, 2:0}, inplace=True) # recode to 0 = no and 1 = yes
data_recoded['dog_exp_4'].replace({1:1, 2:0}, inplace=True) # recode to 0 = no and 1 = yes
data_recoded['dog_exp_5'].replace({1:1, 2:0}, inplace=True) # recode to 0 = no and 1 = yes

data_recoded.head()

first QCAE item:  4
last QCAE item:  34


,pn_code,age,gender,handedness,QCAE_1,QCAE_2,QCAE_3,QCAE_4,QCAE_5,QCAE_6,...,QCAE_31,dog_exp_1,dog_exp_2,dog_exp_3,dog_exp_3_opt1,dog_exp_4,dog_exp_4_opt1,dog_exp_5,dog_exp_5_opt1,dog_exp_5_opt2
0,EV01,19,0,1,3,3,3,4,3,3,...,3,4,3,0,NaN,0,NaN,0,NaN,NaN
1,EV02,21,1,1,4,3,4,3,3,3,...,2,3,2,0,NaN,0,NaN,0,NaN,NaN
2,EV03,26,1,1,4,2,4,4,4,4,...,4,4,3,0,NaN,0,NaN,0,NaN,NaN
3,EV04,20,0,1,3,3,3,3,3,2,...,3,5,4,1,20.0,1,1.0,1,3.0,Hundeschule
4,EV05,20,0,1,4,3,3,4,4,2,...,3,4,3,0,NaN,0,NaN,0,NaN,NaN


Create a new data frame only including the variables relevant for analysis 

In [8]:
# iniitiate dataframe
data = pd.DataFrame()

# add participant_id
data['participant_id'] = [f"sub-{i:02d}" for i in range(1, 41)]
data.head()

,participant_id
0,sub-01
1,sub-02
2,sub-03
3,sub-04
4,sub-05


In [9]:
# keep sex and age variable
data['age'] = data_recoded['age']
data['sex'] = data_recoded['gender']
data['handedness'] = data_recoded['handedness']

# calculate QCAE subscale scores
data['perspective_taking'] = data_recoded.iloc[:, [18,19,22,23,24,25,27,28,29,30]].mean(axis=1)
data['online_simulation'] = data_recoded.iloc[:, [4,6,7,8,9,21,31,33,34]].mean(axis=1)
data['emotion_contagion'] = data_recoded.iloc[:, [11,12,16,17]].mean(axis=1)
data['proximal_responsivity'] = data_recoded.iloc[:, [10,13,15,26]].mean(axis=1)
data['peripheral_responsivity'] = data_recoded.iloc[:, [5,14,20,32]].mean(axis=1)

# calculate QCAE higher order factors
data['cognitive_empathy'] = data.loc[:, ['perspective_taking','online_simulation']].mean(axis=1)
data['affective_empathy'] = data.loc[:, ['emotion_contagion','proximal_responsivity','peripheral_responsivity']].mean(axis=1)

# dog expertise questionnaire
data['affection'] = data_recoded['dog_exp_1']
data['knowledge'] = data_recoded['dog_exp_2']
data['shared_household'] = data_recoded['dog_exp_3']
data['shared_household_years'] = data_recoded['dog_exp_3_opt1']
data['ownership'] = data_recoded['dog_exp_4']
data['ownership_count'] = data_recoded['dog_exp_4_opt1']
data['expertise'] = data_recoded['dog_exp_5']
data['expertise_years'] = data_recoded['dog_exp_5_opt1']
data['expertise_open'] = data_recoded['dog_exp_5_opt2']

# replace handedness = 1 with R for right-handed
#data.loc[data['handedness'] == 1, 'handedness'] = "R"

# create a new dataframe with the variable "dog_expert" when participants answered with yes either on household, ownership, or expertise
dog_expert = pd.DataFrame()
dog_expert['dog_expert'] = np.arange(0,40) # initiate the variable
for i, sbj in enumerate(data['participant_id']):
    if data['shared_household'][i] == 1 or data['ownership'][i] == 1 or data['expertise'][i] == 1:
        dog_expert['dog_expert'][i] = 1
    else:
        dog_expert['dog_expert'][i] = 0

# insert the dog expertise variable to our og dataframe
data.insert(3, "dog_expert", dog_expert)

# replace missing values with "n/a"
data = data.replace(r'^\s*$', np.nan, regex=True)
  
data

,participant_id,age,sex,dog_expert,handedness,perspective_taking,online_simulation,emotion_contagion,proximal_responsivity,peripheral_responsivity,...,affective_empathy,affection,knowledge,shared_household,shared_household_years,ownership,ownership_count,expertise,expertise_years,expertise_open
0,sub-01,19,0,0,1,3.4,3.111111,3.25,3.50,3.25,...,3.333333,4,3,0,NaN,0,NaN,0,NaN,NaN
1,sub-02,21,1,0,1,2.3,3.333333,2.50,2.00,2.50,...,2.333333,3,2,0,NaN,0,NaN,0,NaN,NaN
2,sub-03,26,1,0,1,4.0,3.888889,2.75,3.00,2.75,...,2.833333,4,3,0,NaN,0,NaN,0,NaN,NaN
3,sub-04,20,0,1,1,3.0,2.777778,2.75,3.25,3.00,...,3.000000,5,4,1,20.0,1,1.0,1,3.0,Hundeschule
4,sub-05,20,0,0,1,3.5,3.333333,3.50,3.00,3.25,...,3.250000,4,3,0,NaN,0,NaN,0,NaN,NaN
5,sub-06,21,0,0,1,3.3,3.555556,3.00,3.00,2.75,...,2.916667,5,3,0,NaN,0,NaN,0,NaN,NaN
6,sub-07,22,1,0,1,3.9,2.555556,3.00,3.50,2.50,...,3.000000,5,3,0,NaN,0,NaN,0,NaN,NaN
7,sub-08,22,0,0,1,3.5,3.000000,2.75,3.25,3.50,...,3.166667,4,3,0,NaN,0,NaN,0,NaN,NaN
8,sub-09,27,0,1,1,3.0,2.555556,3.25,3.75,3.50,...,3.500000,4,3,1,3.0,0,NaN,1,1.0,Familienhund als Welpe trainiert und Begleitun...
9,sub-10,25,0,1,1,2.6,3.111111,2.50,4.25,1.50,...,2.750000,5,3,1,8.0,1,2.0,0,NaN,NaN


### Variable descriptives

In [10]:
data.describe()

,age,sex,dog_expert,handedness,perspective_taking,online_simulation,emotion_contagion,proximal_responsivity,peripheral_responsivity,cognitive_empathy,affective_empathy,affection,knowledge,shared_household,shared_household_years,ownership,ownership_count,expertise,expertise_years
count,40.000000,40.000000,40.000000,40.0,40.0000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,17.000000,40.000000,7.000000,40.000000,6.000000
mean,23.000000,0.450000,0.425000,1.0,3.2800,3.286111,2.818750,3.093750,2.843750,3.283056,2.918750,4.000000,3.175000,0.425000,8.382353,0.175000,1.142857,0.175000,2.083333
std,2.611611,0.503831,0.500641,0.0,0.5095,0.382332,0.515738,0.562411,0.657032,0.379819,0.448362,0.816497,0.812956,0.500641,7.114091,0.384808,0.377964,0.384808,1.114301
min,19.000000,0.000000,0.000000,1.0,2.1000,2.555556,1.000000,1.750000,1.000000,2.577778,1.750000,2.000000,2.000000,0.000000,0.500000,0.000000,1.000000,0.000000,0.500000
25%,21.000000,0.000000,0.000000,1.0,3.0000,3.083333,2.500000,2.750000,2.500000,2.955556,2.729167,4.000000,3.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.250000
50%,23.000000,0.000000,0.000000,1.0,3.4000,3.333333,2.750000,3.250000,2.875000,3.277778,3.000000,4.000000,3.000000,0.000000,8.000000,0.000000,1.000000,0.000000,2.500000
75%,25.000000,1.000000,1.000000,1.0,3.7000,3.555556,3.000000,3.312500,3.500000,3.541667,3.166667,5.000000,4.000000,1.000000,13.000000,0.000000,1.000000,0.000000,3.000000
max,28.000000,1.000000,1.000000,1.0,4.0000,4.000000,4.000000,4.250000,3.750000,3.944444,3.833333,5.000000,5.000000,1.000000,20.000000,1.000000,2.000000,1.000000,3.000000


### Summary / descriptive stats

In [11]:
# Sex
print("Number of female paricitpants =", np.sum(data['sex']==0))
print("Number of male paricitpants =", np.sum(data['sex']==1))
print("Percentage of woman =", np.sum(data['sex']==0)*100/40)

# dog expertise
print("Number of dog experts:", np.sum(data['dog_expert']==1))
print("Number of non-dog experts:", np.sum(data['dog_expert']==0))

Number of female paricitpants = 22
Number of male paricitpants = 18
Percentage of woman = 55.0
Number of dog experts: 17
Number of non-dog experts: 23


Correlations

In [13]:
corr = data.corr(method='spearman') # spearman correlation as many variables are categoricals
corr.style.background_gradient(cmap='GnBu').set_precision(2)

/tmp/ipykernel_1755763/2126825084.py:2: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  corr.style.background_gradient(cmap='GnBu').set_precision(2)
/usr/local/lib/python3.8/dist-packages/pandas/io/formats/style.py:2813: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
/usr/local/lib/python3.8/dist-packages/pandas/io/formats/style.py:2814: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


,age,sex,dog_expert,handedness,perspective_taking,online_simulation,emotion_contagion,proximal_responsivity,peripheral_responsivity,cognitive_empathy,affective_empathy,affection,knowledge,shared_household,shared_household_years,ownership,ownership_count,expertise,expertise_years
age,1.00,0.16,0.26,nan,0.04,-0.05,-0.17,-0.14,-0.20,-0.01,-0.22,-0.00,0.08,0.26,-0.01,0.02,0.20,-0.04,0.15
sex,0.16,1.00,-0.17,nan,-0.09,-0.19,0.02,-0.14,-0.24,-0.18,-0.18,-0.18,0.03,-0.17,0.29,-0.28,-0.17,-0.15,0.22
dog_expert,0.26,-0.17,1.00,nan,-0.12,0.06,-0.32,-0.18,0.03,-0.00,-0.18,0.22,0.50,1.00,nan,0.54,nan,0.54,nan
handedness,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
perspective_taking,0.04,-0.09,-0.12,nan,1.00,0.50,0.09,-0.06,0.10,0.88,0.07,0.11,-0.00,-0.12,0.06,-0.15,-0.41,-0.00,0.56
online_simulation,-0.05,-0.19,0.06,nan,0.50,1.00,0.02,-0.11,0.20,0.82,0.07,0.03,-0.12,0.06,-0.29,-0.12,-0.10,-0.14,0.34
emotion_contagion,-0.17,0.02,-0.32,nan,0.09,0.02,1.00,0.56,0.44,0.02,0.80,0.13,-0.02,-0.32,0.04,-0.26,-0.32,0.00,0.10
proximal_responsivity,-0.14,-0.14,-0.18,nan,-0.06,-0.11,0.56,1.00,0.36,-0.10,0.75,0.05,-0.16,-0.18,-0.26,-0.13,0.64,-0.03,-0.18
peripheral_responsivity,-0.20,-0.24,0.03,nan,0.10,0.20,0.44,0.36,1.00,0.17,0.79,-0.04,-0.16,0.03,-0.65,-0.16,-0.62,0.11,-0.36
cognitive_empathy,-0.01,-0.18,-0.00,nan,0.88,0.82,0.02,-0.10,0.17,1.00,0.05,0.07,-0.02,-0.00,-0.13,-0.14,-0.41,-0.05,0.58


**Dog experts vs. non experts**

In [15]:
df_experts = data[data['dog_expert'] == 1]
df_nonexperts = data[data['dog_expert'] == 0]

In [16]:
print("Number of female expert participants =", np.sum(df_experts['sex']==0))
print("Number of male expert participants =", np.sum(df_experts['sex']==1))
print("Percentage of expert woman =", np.sum(df_experts['sex']==0)*100/len(df_experts))
df_experts.describe()

Number of female expert participants = 11
Number of male expert participants = 6
Percentage of expert woman = 64.70588235294117


,age,sex,dog_expert,handedness,perspective_taking,online_simulation,emotion_contagion,proximal_responsivity,peripheral_responsivity,cognitive_empathy,affective_empathy,affection,knowledge,shared_household,shared_household_years,ownership,ownership_count,expertise,expertise_years
count,17.000000,17.000000,17.0,17.0,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.0,17.000000,17.000000,7.000000,17.000000,6.000000
mean,23.705882,0.352941,1.0,1.0,3.217647,3.320261,2.602941,2.970588,2.882353,3.268954,2.818627,4.176471,3.647059,1.0,8.382353,0.411765,1.142857,0.411765,2.083333
std,2.687115,0.492592,0.0,0.0,0.535298,0.382653,0.538073,0.624264,0.613122,0.416704,0.432246,0.882843,0.996317,0.0,7.114091,0.507300,0.377964,0.507300,1.114301
min,19.000000,0.000000,1.0,1.0,2.100000,2.555556,1.000000,1.750000,1.500000,2.605556,1.750000,2.000000,2.000000,1.0,0.500000,0.000000,1.000000,0.000000,0.500000
25%,22.000000,0.000000,1.0,1.0,3.000000,3.111111,2.500000,2.500000,2.500000,2.888889,2.666667,4.000000,3.000000,1.0,1.000000,0.000000,1.000000,0.000000,1.250000
50%,23.000000,0.000000,1.0,1.0,3.400000,3.333333,2.750000,3.250000,3.000000,3.477778,2.916667,4.000000,4.000000,1.0,8.000000,0.000000,1.000000,0.000000,2.500000
75%,26.000000,1.000000,1.0,1.0,3.700000,3.555556,3.000000,3.250000,3.500000,3.566667,3.083333,5.000000,4.000000,1.0,13.000000,1.000000,1.000000,1.000000,3.000000
max,28.000000,1.000000,1.0,1.0,3.900000,4.000000,3.250000,4.250000,3.500000,3.894444,3.500000,5.000000,5.000000,1.0,20.000000,1.000000,2.000000,1.000000,3.000000


In [17]:
print("Number of female non-expert participants =", np.sum(df_nonexperts['sex']==0))
print("Number of male non-expert paricitpants =", np.sum(df_nonexperts['sex']==1))
print("Percentage of non-expert woman =", np.sum(df_nonexperts['sex']==0)*100/len(df_nonexperts))
df_nonexperts.describe()

Number of female non-expert participants = 11
Number of male non-expert paricitpants = 12
Percentage of non-expert woman = 47.82608695652174


,age,sex,dog_expert,handedness,perspective_taking,online_simulation,emotion_contagion,proximal_responsivity,peripheral_responsivity,cognitive_empathy,affective_empathy,affection,knowledge,shared_household,shared_household_years,ownership,ownership_count,expertise,expertise_years
count,23.000000,23.000000,23.0,23.0,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.0,0.0,23.0,0.0,23.0,0.0
mean,22.478261,0.521739,0.0,1.0,3.326087,3.260870,2.978261,3.184783,2.815217,3.293478,2.992754,3.869565,2.826087,0.0,NaN,0.0,NaN,0.0,NaN
std,2.483941,0.510754,0.0,0.0,0.496549,0.388685,0.445387,0.506870,0.699908,0.359424,0.454990,0.757049,0.387553,0.0,NaN,0.0,NaN,0.0,NaN
min,19.000000,0.000000,0.0,1.0,2.300000,2.555556,2.250000,2.000000,1.000000,2.577778,2.166667,2.000000,2.000000,0.0,NaN,0.0,NaN,0.0,NaN
25%,21.000000,0.000000,0.0,1.0,3.100000,3.055556,2.750000,3.000000,2.500000,3.033333,2.833333,3.500000,3.000000,0.0,NaN,0.0,NaN,0.0,NaN
50%,22.000000,1.000000,0.0,1.0,3.400000,3.333333,3.000000,3.250000,2.750000,3.266667,3.000000,4.000000,3.000000,0.0,NaN,0.0,NaN,0.0,NaN
75%,23.000000,1.000000,0.0,1.0,3.750000,3.555556,3.125000,3.500000,3.250000,3.480556,3.291667,4.000000,3.000000,0.0,NaN,0.0,NaN,0.0,NaN
max,28.000000,1.000000,0.0,1.0,4.000000,3.888889,4.000000,4.000000,3.750000,3.944444,3.833333,5.000000,3.000000,0.0,NaN,0.0,NaN,0.0,NaN


In [ ]:
rp.ttest(group1= data['affection'][data['dog_expert'] == 1], group1_name= "Experts",
         group2= data['affection'][data['dog_expert'] == 0], group2_name= "Non-experts")

/usr/local/lib/python3.10/dist-packages/researchpy/ttest.py:38: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)


(      Variable     N      Mean        SD        SE  95% Conf.  Interval
 0      Experts  17.0  4.176471  0.882843  0.214121   3.722555  4.630387
 1  Non-experts  23.0  3.869565  0.757049  0.157856   3.542193  4.196938
 2     combined  40.0  4.000000  0.816497  0.129099   3.738872  4.261128,
                       Independent t-test  results
 0  Difference (Experts - Non-experts) =    0.3069
 1                  Degrees of freedom =   38.0000
 2                                   t =    1.1811
 3               Two side test p value =    0.2449
 4              Difference < 0 p value =    0.8776
 5              Difference > 0 p value =    0.1224
 6                           Cohen's d =    0.3778
 7                           Hedge's g =    0.3703
 8                      Glass's delta1 =    0.3476
 9                    Point-Biserial r =    0.1882)

In [ ]:
rp.ttest(group1= data['knowledge'][data['dog_expert'] == 1], group1_name= "Experts",
         group2= data['knowledge'][data['dog_expert'] == 0], group2_name= "Non-experts")

/usr/local/lib/python3.10/dist-packages/researchpy/ttest.py:38: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)


(      Variable     N      Mean        SD        SE  95% Conf.  Interval
 0      Experts  17.0  3.647059  0.996317  0.241642   3.134800  4.159318
 1  Non-experts  23.0  2.826087  0.387553  0.080810   2.658496  2.993678
 2     combined  40.0  3.175000  0.812956  0.128540   2.915004  3.434996,
                       Independent t-test  results
 0  Difference (Experts - Non-experts) =    0.8210
 1                  Degrees of freedom =   38.0000
 2                                   t =    3.6123
 3               Two side test p value =    0.0009
 4              Difference < 0 p value =    0.9996
 5              Difference > 0 p value =    0.0004
 6                           Cohen's d =    1.1554
 7                           Hedge's g =    1.1324
 8                      Glass's delta1 =    0.8240
 9                    Point-Biserial r =    0.5056)

### Create a participants.tsv file

In [12]:
data.to_csv(data_dir / "participants.tsv", sep='\t',na_rep='n/a', index=False)

### Create a BIDS style dataset_description.json file 


In [24]:
import json

Now we can create a dictionary, where we store information on participants. 

In [28]:
dataset_dict = {
    "Name": "Action Observation and Action Task, Master Thesis",
    "Authors": ["Olaf Borghi", "Magdalena Boch"],
    "Funding": ["This project was supported by the Austrian Science Fund (FWF): W1262-B29 and by the Vienna Science and Technology Fund (WWTF), the City of Vienna and ithuba Capital AG through project CS18-012 , and the Messerli Foundation (S\u00f6renberg, Switzerland). The funders had no role in study design, data collection and analysis, decision to publish, or preparation of the manuscript."], 
    "BIDSVersion": "1.2.1", 
    "Date": "2021"
}

From the dictionary, we can now create a .json file named dataset_description.

In [29]:
# create the json file from the dict and store it in the wd
jsonString = json.dumps(dataset_dict,
                        sort_keys=False,
                        indent=4,
                        separators=(',', ': '))
jsonFile = open("dataset_description.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

In [30]:
%%bash
cat dataset_description.json

{
    "Name": "Action Observation and Action Task, Master Thesis",
    "Authors": [
        "Olaf Borghi",
        "Magdalena Boch"
    ],
    "Funding": [
        "This project was supported by the Austrian Science Fund (FWF): W1262-B29 and by the Vienna Science and Technology Fund (WWTF), the City of Vienna and ithuba Capital AG through project CS18-012 , and the Messerli Foundation (S\u00f6renberg, Switzerland). The funders had no role in study design, data collection and analysis, decision to publish, or preparation of the manuscript."
    ],
    "BIDSVersion": "1.2.1",
    "Date": "2021"
}

### Create a BIDS style descriptives file (participants.json)

In [34]:
participant_dict = {
    "age": {
        "Description": "Age of the participant", 
        "Units": "years"
    }, 
    "sex": {
        "Description": "Sex of the participant", 
        "Levels": {
            "0": "Female", 
            "1": "Male",
            "2": "Other",
        }
    },
    "dog_expert": {
        "Description": "A person that either lived in a household with dogs, owned one or several dogs, or has other significant expertise (e.g., from training or working with dogs)",
        "Levels": {
            "0": "Non-expert",
            "1": "Expert"
        }
    },
    "handedness": {
        "Description": "Dominant hand of the participant", 
        "Levels": {
            "R": "right-handed"
        }
    }, 
    "perspective_taking": {
        "Description": "the ability to interpret situations from the perspective of others", 
        "Questionnaire": "Questionnaire of Cognitive and Affective Empathy (Reniers et al., 2016)",
        "Units": "arbitrary interval scale ranging from 1 to 4"
    }, 
    "online_simulation": {
        "Description": "the ability to put oneself in another person’s place to understand and anticipate their feelings", 
        "Questionnaire": "Questionnaire of Cognitive and Affective Empathy (Reniers et al., 2016)",
        "Units": "arbitrary interval scale ranging from 1 to 4"
    }, 
    "emotion_contagion": {
        "Description": "the tendency to automatically take over others’ emotions", 
        "Questionnaire": "Questionnaire of Cognitive and Affective Empathy (Reniers et al., 2016)",
        "Units": "arbitrary interval scale ranging from 1 to 4"
    }, 
    "proximal_responsivity": {
        "Description": "the affective responsivity when observing others’ emotions in a more close social context", 
        "Questionnaire": "Questionnaire of Cognitive and Affective Empathy (Reniers et al., 2016)",
        "Units": "arbitrary interval scale ranging from 1 to 4"
    }, 
    "peripheral_responsivity": {
        "Description": "the affective responsivity when observing others’ emotions in a more detached context, e.g., when watching characters in a movie", 
        "Questionnaire": "Questionnaire of Cognitive and Affective Empathy (Reniers et al., 2016)",
        "Units": "arbitrary interval scale ranging from 1 to 4"
    }, 
    "cognitive_empathy": {
        "Description": "the ability to construct a working model of the emotional states of others", 
        "Questionnaire": "Questionnaire of Cognitive and Affective Empathy (Reniers et al., 2016)",
        "Units": "arbitrary interval scale from 1 to 4"
    }, 
    "affective_empathy": {
        "Description": "the ability to be sensitive to and vicariously experience the feelings of others", 
        "Questionnaire": "Questionnaire of Cognitive and Affective Empathy (Reniers et al., 2016)",
        "Units": "arbitrary interval scale ranging from 1 to 4"
    }, 
    "affection": {
        "Description": "How much do you like dogs?", 
        "Levels": {
            "1": "Not at all", 
            "2": "A little", 
            "3": "Some", 
            "4": "Very much", 
            "5": "Extremely much"
        }
    }, 
    "knowledge": {
        "Description": "How much do you know about dog behaviour?", 
        "Levels": {
            "1": "Not at all", 
            "2": "A little", 
            "3": "Some", 
            "4": "Very much", 
            "5": "Extremely much"
        }
    }, 
    "shared_household": {
        "Description": "Have you ever shared or currently share a household with at least one dog?", 
        "Levels": {
            "1": "yes", 
            "0": "no"
        }
    }, 
    "shared_household_years": {
        "Description": "Time spent living with at least one dog in same household", 
        "Units": "years"
    }, 
    "ownership": {
        "Description": "Do you currently own a dog?", 
        "Levels": {
            "1": "yes", 
            "0": "no"
        }
    }, 
    "ownership_count": {
        "Description": "How many dogs do you currently own?", 
        "Units": "count"
    }, 
    "expertise": {
        "Description": "Do you have any experience in dog training or hobbies involving working with dogs (e.g. agility?)", 
        "Levels": {
            "1": "yes", 
            "0": "no"
        }
    }, 
    "expertise_years": {
        "Description": "If expertise was answered with yes: For how many years?", 
        "Unit": "years"
    }, 
    "expertise_open": {
        "Description": "if expertise was answered with yes: What kind of experience do you have?", 
        "Unit": "string"
    }
}

In [35]:
# create the json file from the dict and store it in the wd
jsonString = json.dumps(participant_dict,
                        sort_keys=False,
                        indent=4,
                        separators=(',', ': '))
jsonFile = open("participants.json", "w")
jsonFile.write(jsonString)
jsonFile.close()

In [36]:
%%bash
cat participants.json

{
    "age": {
        "Description": "Age of the participant",
        "Units": "years"
    },
    "sex": {
        "Description": "Sex of the participant",
        "Levels": {
            "0": "Female",
            "1": "Male",
            "2": "Other"
        }
    },
    "dog_expert": {
        "Description": "A person that either lived in a household with dogs, owned one or several dogs, or has other significant expertise (e.g., from training or working with dogs)",
        "Levels": {
            "0": "Non-expert",
            "1": "Expert"
        }
    },
    "handedness": {
        "Description": "Dominant hand of the participant",
        "Levels": {
            "R": "right-handed"
        }
    },
    "perspective_taking": {
        "Description": "the ability to interpret situations from the perspective of others",
        "Questionnaire": "Questionnaire of Cognitive and Affective Empathy (Reniers et al., 2016)",
        "Units": "arbitrary interval scale ranging from 1 to 